In [20]:
import os
import numpy as np
import cv2
from scipy.ndimage import sobel
import pandas as pd

In [21]:
def phase_congruency(image):
    grad_x = sobel(image, axis=0)
    grad_y = sobel(image, axis=1)
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    grad_orientation = np.arctan2(grad_y, grad_x)
    return grad_magnitude * np.abs(np.cos(grad_orientation))

In [22]:
def gradient_magnitude(image):
    grad_x = sobel(image, axis=0)
    grad_y = sobel(image, axis=1)
    return np.sqrt(grad_x**2 + grad_y**2)

In [23]:
def compute_fsim(original, denoised):
    pc_orig = phase_congruency(original)
    pc_denoised = phase_congruency(denoised)
    
    grad_orig = gradient_magnitude(original)
    grad_denoised = gradient_magnitude(denoised)
    
    pc_orig /= np.max(pc_orig)
    pc_denoised /= np.max(pc_denoised)
    
    grad_orig /= np.max(grad_orig)
    grad_denoised /= np.max(grad_denoised)
    
    pc_similarity = np.sum(np.abs(pc_orig - pc_denoised)) / np.sum(np.abs(pc_orig + pc_denoised))
    grad_similarity = np.sum(np.abs(grad_orig - grad_denoised)) / np.sum(np.abs(grad_orig + grad_denoised))
    
    fsim_value = 1 - 0.5 * (pc_similarity + grad_similarity)
    
    return fsim_value

In [24]:
def evaluate_fsim(original_folder, denoised_folder):
    original_files = sorted(os.listdir(original_folder))
    denoised_files = sorted(os.listdir(denoised_folder))

    results = []

    for orig_file, den_file in zip(original_files, denoised_files):
        orig_path = os.path.join(original_folder, orig_file)
        den_path = os.path.join(denoised_folder, den_file)
        
        original = cv2.imread(orig_path, cv2.IMREAD_GRAYSCALE)
        denoised = cv2.imread(den_path, cv2.IMREAD_GRAYSCALE)

        if original is None or denoised is None:
            print(f"Could not read {orig_file} or {den_file}. Skipping...")
            continue
            
        fsim_value = compute_fsim(original, denoised)
        results.append({"Original Image": orig_file, "Denoised Image": den_file, "FSIM": fsim_value})

    results_df = pd.DataFrame(results)

    avg_fsim = results_df["FSIM"].mean()
    print(f"\nAverage FSIM: {avg_fsim:.4f}")
    
    return results_df, avg_fsim

In [25]:
original_folder = "../Image_Dataset/Original_Images"
denoised_folder = "../Image_Dataset/FastNLM_Speckle_Images"

results_df, avg_fsim = evaluate_fsim(original_folder, denoised_folder)
display(results_df)

Could not read .ipynb_checkpoints or .ipynb_checkpoints. Skipping...


C:\Users\84835\AppData\Local\Temp\ipykernel_6944\1783377901.py:14: RuntimeWarning: invalid value encountered in scalar divide
  pc_similarity = np.sum(np.abs(pc_orig - pc_denoised)) / np.sum(np.abs(pc_orig + pc_denoised))
C:\Users\84835\AppData\Local\Temp\ipykernel_6944\1783377901.py:15: RuntimeWarning: invalid value encountered in scalar divide
  grad_similarity = np.sum(np.abs(grad_orig - grad_denoised)) / np.sum(np.abs(grad_orig + grad_denoised))



Average FSIM: nan


,Original Image,Denoised Image,FSIM
0,Image_1.jpg,FastNLM_SpeckeNoise_Image_1.jpg,NaN
1,Image_2.jpg,FastNLM_SpeckeNoise_Image_2.jpg,NaN
2,Image_3.jpg,FastNLM_SpeckeNoise_Image_3.jpg,NaN
3,Image_4.jpg,FastNLM_SpeckeNoise_Image_4.jpg,NaN
4,Image_5.jpg,FastNLM_SpeckeNoise_Image_5.jpg,NaN
5,Image_6.jpg,FastNLM_SpeckeNoise_Image_6.jpg,NaN
6,Image_7.jpg,FastNLM_SpeckeNoise_Image_7.jpg,NaN
7,Image_8.jpg,FastNLM_SpeckeNoise_Image_8.jpg,NaN
